[Toc]


# 保存和恢复模型

- 配置
    - 安装并导入
    - 获取实例数据集
    - 定义模型
- 在训练期间保存模型（以checkpoints形式保存）
    - Checkpoint回调用法
    - checkpoint回调选项
- 这些文件是什么
- 手动保存权重
- 保存整个模型
    - 新的高级.keras格式
    - SavedModel格式



## 配置

### 安装并导入Tensorflow和依赖项

```pip
pip install pyyaml h5py
```


In [1]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.13.0


## 获取实例数据集

为了演示如何保存和加载权重，您将使用MNIST数据集，为了加快运行速度，使用前1000个样本

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28*28) / 255.0
test_images = test_images[:1000].reshape(-1, 28*28) / 255.0

11490434/11490434 [==============================] - 3s 0us/step


### 定义模型

首先构建一个简单的序列（sequential）模型

In [3]:
def create_model():
    model = tf.keras.Sequential([
        keras.layers.Dense(512, activation='elu', input_shape=(784, )),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])

    model.compile(
        optimizer='adam',
        # from_logits=False表示输出的logits需要经过激活函数的处理，默认为False。
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
    )

    return model

model = create_model()

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## 在训练期间保存模型（以Checkpoints形式保存）

您可以使用经过训练的模型而无需重新训练，或者在训练过程中中断的情况下从离开处继续训练，tf.keras.callbacks.ModelCheckpoint回调允许您在训练期间和结束时持续保存模型

### Check回调用法

创建一个旨在训练期间保存权重的tf.keras.callbacks.ModelCheckpoint回调：

In [5]:
checkpoint_path = "./training-1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)

model.fit(train_images, train_labels, epochs=10, validation_data=(test_images, test_labels), callbacks=[cp_callback])

Epoch 1/10
29/32 [==========================>...] - ETA: 0s - loss: 0.0478 - sparse_categorical_accuracy: 0.9968
Epoch 1: saving model to ./training-1\cp.ckpt
32/32 [==============================] - 0s 6ms/step - loss: 0.0470 - sparse_categorical_accuracy: 0.9970 - val_loss: 0.5232 - val_sparse_categorical_accuracy: 0.8450
Epoch 2/10
28/32 [=========================>....] - ETA: 0s - loss: 0.0357 - sparse_categorical_accuracy: 0.9989
Epoch 2: saving model to ./training-1\cp.ckpt
32/32 [==============================] - 0s 6ms/step - loss: 0.0384 - sparse_categorical_accuracy: 0.9970 - val_loss: 0.5588 - val_sparse_categorical_accuracy: 0.8380
Epoch 3/10
26/32 [=======================>......] - ETA: 0s - loss: 0.0394 - sparse_categorical_accuracy: 0.9964
Epoch 3: saving model to ./training-1\cp.ckpt
32/32 [==============================] - 0s 6ms/step - loss: 0.0372 - sparse_categorical_accuracy: 0.9970 - val_loss: 0.5912 - val_sparse_categorical_accuracy: 0.8260
Epoch 4/10
28/32 [====

这将创建一个Tensorflow checkpoint文件集合，这些文件在每个epoch结束时更新

In [6]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.data-00000-of-00001', 'cp.ckpt.index']

只要两个模型共享相同的架构，您就可以在他们之间共享权重，因此当从仅权重恢复模型时，创建一个与原始模型具有相同架构的模型，然后设置其权重

现在，重新构建一个未经训练的全新模型并基于测试集对其进行评估，未经训练的模型将以机会水平执行（约10%的准确率）

In [7]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels, verbose=2)

print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3298 - sparse_categorical_accuracy: 0.1320 - 83ms/epoch - 3ms/step
Untrained model, accuracy: 13.20%


然后从checkpoint加载权重并重新评估

In [8]:
model.load_weights(checkpoint_path)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)

print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.6148 - sparse_categorical_accuracy: 0.8390 - 38ms/epoch - 1ms/step
Untrained model, accuracy: 83.90%


### checkpoint回调选项

回调提供了几个选项，为checkpoint提供唯一名称并调整checkpoint频率

训练一个新模型，每五个epochs保存一次唯一命名的checkpoint

In [9]:
checkpoint_path = "./training-2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

import math
n_batches = len(train_images) / batch_size
n_batches = math.ceil(n_batches)  # 舍弃小数部分。向整数部分进1

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*n_batches
)

model = create_model()

model.save_weights(checkpoint_path.format(epoch=0))
# batch_size: 批次寸，即训练一次网络所用的样本数
model.fit(train_images, train_labels, epochs=50, batch_size=batch_size, callbacks=[cp_callback], validation_data=(test_images, test_labels), verbose=0)


Epoch 5: saving model to ./training-2\cp-0005.ckpt

Epoch 10: saving model to ./training-2\cp-0010.ckpt

Epoch 15: saving model to ./training-2\cp-0015.ckpt

Epoch 20: saving model to ./training-2\cp-0020.ckpt

Epoch 25: saving model to ./training-2\cp-0025.ckpt

Epoch 30: saving model to ./training-2\cp-0030.ckpt

Epoch 35: saving model to ./training-2\cp-0035.ckpt

Epoch 40: saving model to ./training-2\cp-0040.ckpt

Epoch 45: saving model to ./training-2\cp-0045.ckpt

Epoch 50: saving model to ./training-2\cp-0050.ckpt


现在，检查生成的检查点并选择最新检查点

In [10]:
os.listdir(checkpoint_dir)

['checkpoint',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.index',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.index',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.index',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.index',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.index',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.index']

In [12]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'./training-2\\cp-0050.ckpt'

注：默认Tensorflow格式只保存最近5个检查点，
要进行测试，请重置模型并加载最新检查点

In [13]:
model = create_model()

model.load_weights(latest)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.7872 - sparse_categorical_accuracy: 0.8380 - 87ms/epoch - 3ms/step
Restored model, accuracy: 83.80%


## 这些文件是什么？

上述代码可将权重存储到检查点格式文件（仅包含二进制格式训练权重）的合集中

检查点包含：
- 一个或多个模型权重的分片
- 一个索引文件，指示哪些权重存储在哪个分片中

如果您在一台计算机上训练模型，您将获得一个具有如下后缀的分片：data-00000-of-00001

## 手动保存权重

要手动保存权重，请使用tf.keras.Model.save-weights

In [14]:
model.save_weights('./checkpoints/my_checkpoint')

model = create_model()

model.load_weights('./checkpoints/my_checkpoint')

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.7872 - sparse_categorical_accuracy: 0.8380 - 91ms/epoch - 3ms/step
Restored model, accuracy: 83.80%


## 保存整个模型

**调用tf.keras.Model.save,将模型的架构，权重和训练配置保存在单个model.keras zip存档中**

整个模型可以保存为三种不同的文件格式（新的 .keras 格式和两种旧格式：SavedModel 和 HDF5）。将模型保存为 path/to/model.keras 会自动以最新格式保存。

保存功能齐全的模型非常有用 - 您可以将它们加载到 TensorFlow.js（SavedModel，HDF5）中，然后在 Web 浏览器中训练和运行它们，或者使用 TensorFlow Lite（保存的模型，HDF5）将它们转换为在移动设备上运行）

In [15]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

model.save('./my_model.keras')

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.0101 - sparse_categorical_accuracy: 0.7100
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4020 - sparse_categorical_accuracy: 0.8870
Epoch 3/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2789 - sparse_categorical_accuracy: 0.9290
Epoch 4/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2227 - sparse_categorical_accuracy: 0.9400
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1676 - sparse_categorical_accuracy: 0.9640


从.keras zip归档重新加载新的keras模型

In [16]:
new_model = tf.keras.models.load_model('./my_model.keras')

new_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               401920    
                                                                 
 dropout_5 (Dropout)         (None, 512)               0         
                                                                 
 dense_11 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


尝试使用加载的模型运行评估个预测

In [17]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)

print('Restored model, accuracy:{:5.2f}'.format(100*acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4907 - sparse_categorical_accuracy: 0.8460 - 80ms/epoch - 3ms/step
Restored model, accuracy:84.60
32/32 [==============================] - 0s 793us/step
(1000, 10)


### SavedModel格式

SavedModel格式是另一种序列化模型的方式，以这种格式保存的模型可以使用tf.keras.models.load_model还原， 并且与Tensorflow Serving兼容

In [18]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

!mkdir -p saved_model
model.save('./saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.0113 - sparse_categorical_accuracy: 0.7160
Epoch 2/5
32/32 [==============================] - 0s 5ms/step - loss: 0.4069 - sparse_categorical_accuracy: 0.8730
Epoch 3/5
32/32 [==============================] - 0s 5ms/step - loss: 0.3010 - sparse_categorical_accuracy: 0.9180
Epoch 4/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2338 - sparse_categorical_accuracy: 0.9400
Epoch 5/5
32/32 [==============================] - 0s 5ms/step - loss: 0.1846 - sparse_categorical_accuracy: 0.9450
INFO:tensorflow:Assets written to: ./saved_model/my_model\assets


INFO:tensorflow:Assets written to: ./saved_model/my_model\assets
